# Utils

## Create vectors from images

In [143]:
import face_recognition

In [144]:
IMAGE = 'target.jpg'

In [145]:
def get_encodings(image):
    image = face_recognition.load_image_file(image) 
    # detect the faces from the images  
    face_locations = face_recognition.face_locations(image) 
    # encode the 128-dimension face encoding for each face in the image 
    face_encodings = face_recognition.face_encodings(image, face_locations) 
    # Display the 128-dimension for each face detected 
    
    return face_encodings

## Communicate with elasticsearch

In [146]:
import requests

### Create index

In [147]:
# Create index
data = { 
  "mappings" : { 
      "properties" : { 
        "name" : { 
          "type" : "keyword" 
        }, 
        "encoding" : { 
          "type" : "dense_vector", 
          "dims" : 128 
        } 
      } 
    } 
}
res = requests.put('http://localhost:9200/faces', json=data)

### A way to add encodings

In [148]:
def index_encoding(encoding, person):
    data = {'name': person, 'encoding': encoding}
    res = requests.post('http://localhost:9200/faces/_doc', json=data)

# Load dataset
Iterate over each picture in `dataset` and store the first found face to elastic with the name matching the file name

In [149]:
import os

In [150]:
for face in os.listdir('dataset'):
    encoding = get_encodings(f'dataset/{face}')[0]
    index_encoding(encoding.tolist(), face.split('.')[0])

## Recognition

In [151]:
from elasticsearch import Elasticsearch, logger, logging

es = Elasticsearch()

es_logger = logger
es_logger.setLevel(logging.ERROR)

In [152]:
def recognize(image):
    encodings = get_encodings(image)

    for encoding in encodings:
        response = es.search(
            index="faces",
            body={
                "size": 1,
                "_source": "name",
                "query": {
                    "script_score": {
                        "query": {"match_all": {}},
                        "script": {
                            "source": "cosineSimilarity(params.encoding, 'encoding')",
                            "params": {"encoding": encoding.tolist()},
                        },
                    }
                },
            },
        )

        for hit in response["hits"]["hits"]:
            if float(hit["_score"]) > 0.93:
                person = hit["_source"]["name"]
                score = hit["_score"]
                print(f"Found {person} ({score})")
            else:
                print("Found unknown face")


In [154]:
TARGET = 'target.jpg'
recognize(TARGET)

Found me (0.9392916)
